In [5]:
import json
import numpy as np
import cv2

path = 'Climbing-Holds-and-Volumes-14/train'

In [6]:
import os
test_path = "Climbing-Holds-and-Volumes-14/train/_annotations.coco.json"
with open(test_path) as f:
    file = json.loads(f.read())
    images = file['images']
    annotations = file['annotations']
    

In [3]:
print(file.keys())
print(file['categories'])

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])
[{'id': 0, 'name': 'holds', 'supercategory': 'none'}, {'id': 1, 'name': '0', 'supercategory': 'holds'}, {'id': 2, 'name': '1', 'supercategory': 'holds'}]


In [7]:
def deleteCategoryID(annotations, category_id):
    new_annotations = []
    id = 0
    for i in range(len(annotations)):
        if int(annotations[i]['category_id']) != category_id:
            annotations[i]['id'] = id
            new_annotations.append(annotations[i])
            id += 1
          
    return new_annotations

In [5]:
# annotations_cleaned = deleteCategoryID(annotations, 2)

In [6]:
# len(annotations_cleaned)

In [7]:
# file['annotations'] = annotations_cleaned
# json_object = json.dumps(file)
# with open("Climbing-Holds-and-Volumes-14/train/_annotations.coco.json", "w") as outfile:
#     outfile.write(json_object)

In [8]:
len(annotations)

120528

In [9]:
annotations[1]

{'id': 1,
 'image_id': 0,
 'category_id': 1,
 'bbox': [479, 110, 146, 110],
 'area': 16060,
 'segmentation': [],
 'iscrowd': 0}

In [10]:
images[0]

{'id': 0,
 'license': 1,
 'file_name': '31_jpg.rf.cf19005c65737bfcea76345769f2ff5e.jpg',
 'height': 400,
 'width': 825,
 'date_captured': '2023-10-13T13:22:06+00:00'}

In [11]:
len(images)

3876

In [8]:
def extract_bbox(image, bbox_coord):
    x, y, w, h = [int(b) for b in bbox_coord]
    return image[y:y+h, x:x+w]


def parse_annotations(annotations):
    img_id_to_annotations = {}

    for a in annotations:
        if a['image_id'] in img_id_to_annotations:
            img_id_to_annotations[a['image_id']].append(a['id'])
        else:
            img_id_to_annotations[a['image_id']] = [a['id']]

    return img_id_to_annotations

In [13]:
img_id_to_annotations = parse_annotations(annotations)

In [9]:
import os
def getBoudingBoxForImage(imageId, img_id_to_annotations, annotations, images, path, saving_dir=None):
    annotation_ids = img_id_to_annotations[imageId]
    img_path =  os.path.join(path, images[imageId]['file_name'])
    image = cv2.imread(img_path)

    for a_id in annotation_ids:
        extracted_img = extract_bbox(image, annotations[a_id]['bbox'])
        if saving_dir:
            try:
                cv2.imwrite(os.path.join(saving_dir, f"{imageId:05d}_{a_id:05d}.png"), extracted_img)
            except:
                x, y, w, h = [int(b) for b in annotations[a_id]['bbox']]
                print(annotations[a_id]['bbox'])
                print(x, y, w, h)
                print(image.shape)
                print(image[y:y+h, x:x+w])
                # print(image)
                # print(annotations[a_id]['bbox'])
                # print(extracted_img)

In [15]:
getBoudingBoxForImage(0, img_id_to_annotations, annotations, images, "Climbing-Holds-and-Volumes-14/train/", saving_dir="extractedUnlabeledDataset")

In [11]:
def extractAllImages(interval, img_id_to_annotations, annotations, images, path = 'ClimbingHoldDetection-15/train', saving_dir=None):
    from tqdm import tqdm
    for i in tqdm(interval):
        getBoudingBoxForImage(i, img_id_to_annotations, annotations, images, path, saving_dir)

In [17]:
# Make sure saving_dir exist
extractAllImages(range(0,10), img_id_to_annotations, annotations, images, path = 'Climbing-Holds-and-Volumes-14/train/', saving_dir="extractedUnlabeledDataset")

100%|██████████| 10/10 [00:00<00:00, 46.95it/s]


In [18]:
# desired_class = 4
# count = 0
# i = 0 
# while count < 10:
#     if annotations[i]['category_id'] == desired_class:
#         print(images[annotations[i]['image_id']])
#         print(annotations[i]['id'])
#         count += 1

#     i+= 1

In [19]:
# 1: Crimp
# 2: Jug
# 3: Pinch
# 4: Pocket
# 5: Sloper

In [2]:
from torchvision.io import read_image

In [24]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.nn.functional import one_hot
import torch
class LabeledImageDataset(Dataset):
    def __init__(self, annotations, img_dir, num_classes, transform=None):
        # self.img_labels = pd.read_csv(annotations_file)
        self.annotations = annotations
        self.img_dir = img_dir
        self.transform = transform
        self.num_classes = num_classes
    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        imageId = self.annotations[idx]['image_id']
        img_path = os.path.join(self.img_dir, f"{imageId:05d}_{idx:05d}.png")
        
        image = read_image(img_path)
        label = self.annotations[idx]['category_id']

        if self.transform:
            image = self.transform(image)

        one_hot_label = one_hot(torch.tensor(label))
        return image, one_hot_label

In [25]:
class UnlabeledImageDataset(Dataset):
    def __init__(self, annotations, img_dir, transform=None):
        # self.img_labels = pd.read_csv(annotations_file)
        self.annotations = annotations
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        imageId = self.annotations[idx]['image_id']
        img_path = os.path.join(self.img_dir, f"{imageId:05d}_{idx:05d}.png")
        
        image = read_image(img_path)

        if self.transform:
            image = self.transform(image)

        return image

In [26]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
img_dir = "extractedLabeledDataset"
transform = transforms.Compose([
    transforms.Resize((64,64))
])
testDataset = UnlabeledImageDataset(annotations, img_dir, transform=transform)

test_dataloader = DataLoader(testDataset, batch_size=2, shuffle=False)

test_images = next(iter(test_dataloader))

/Users/yanhuali/opt/anaconda3/envs/jupyter/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [27]:
test_images.shape

torch.Size([2, 3, 64, 64])

In [28]:
import torch
from copy import deepcopy
from torch.nn.functional import one_hot
def augmentAndPredict(model, images , k, num_classes, transforms):
    """
    model: model
    images: a batch of images
    k: k transforms
    transforms: a list of transforms legnth k
    """
    batch_size, dims, height, width = images.shape
    all_augmented_inputs = torch.zeros([k*batch_size, dims, height, width])
    all_preds =  torch.zeros([k*batch_size, num_classes])
    for t in range(k):
        transformed_images = transforms[t](images)
        preds = model(transformed_images)
        all_augmented_inputs[t*batch_size:(t+1)*batch_size] = transformed_images
        all_preds[t*batch_size:(t+1)*batch_size] = preds
    
    return all_augmented_inputs, all_preds

